In [125]:
import numpy as np
import os
from PIL import Image, ImageOps
from matplotlib import pyplot as plt
from scipy import signal

In [126]:
path = '/Users/harrisonward/Desktop/CS/Git/pixelator/assets'
gscale1 = "$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. "

In [127]:
gs_dict = {index:value for (index, value) in enumerate(gscale1)}

In [128]:
# store lazily rendered images
images = []
for file in os.listdir(path):
    fname, ftype = file.split('.')
    if fname != '' and ftype == 'jpeg':
        print(file)
        images.append(Image.open(f'{path}/{file}'))

IMG_2012.jpeg
IMG_8737.jpeg
IMG_8740.jpeg
_DSC6014.jpeg
FullSizeRender.jpeg


In [129]:
def avg_brightness(image_array):
    try:
        return int(np.average(image_array, axis=None))
    except ValueError:
        return 0

In [130]:
def convert_to_gs(image_array):
    return np.mean(np.asarray(image_array), axis=2)
    

Find the image array dimensions

In [131]:
def img_to_ascii(image, n_cols, scale):
    output_image = []

    image_array = convert_to_gs(np.array(image)).T

    W, H = image_array.shape

    # tile width
    w = W / n_cols

    # tile height 
    h = w / scale

    n_rows = int(H/h)

    print(f"cols: {n_cols}, rows: {n_rows}")
    print(f"tile dims: {w} x {h}")

    for i in range(n_rows):
        
        # pixel number times tile height
        y1 = int(i * h)
        
        # pixel one to the right
        # could probably add a % to handle the wrap around
        y2 = int((i+1)*h)

        if i == n_rows - 1:
            y2 = H

        # add an empty string 
        output_image.append("")

        for j in range(n_cols):
            # x tile is the pixel number times the tile width
            x1 = int(j*w)
            x2 = int((j+1)*w)

            if i == n_cols - 1:
                x2 = W
            
            # crop the image, get the tile 
            # could also take a slice of the matrix at these indicies
            tile = image_array[x1:x2, y1:y2]

            # get the average brightness
            avg = avg_brightness(tile)

            # look up and append the char 
            output_image[i] += gs_dict[int((avg*69)/255)]
    
    return output_image


In [172]:
def img_to_ascii_to_txt(image, n_cols, scale, output):
    f = open(output, 'w')

    image_array = convert_to_gs(np.array(image)).T

    W, H = image_array.shape

    # tile width
    w = W / n_cols

    # tile height 
    h = w / scale

    n_rows = int(H/h)

    print(f"cols: {n_cols}, rows: {n_rows}")
    print(f"tile dims: {w} x {h}")

    for i in range(n_rows):
        line = ''
        
        # pixel number times tile height
        y1 = int(i * h)
        
        # pixel one to the right
        # could probably add a % to handle the wrap around
        y2 = int((i+1)*h)

        if i == n_rows - 1:
            y2 = H

        for j in range(n_cols):
            # x tile is the pixel number times the tile width
            x1 = int(j*w)
            x2 = int((j+1)*w)

            if i == n_cols - 1:
                x2 = W
            
            # crop the image, get the tile 
            # could also take a slice of the matrix at these indicies
            tile = image_array[x1:x2, y1:y2]

            # get the average brightness
            avg = avg_brightness(tile)

            # look up and append the char 
            line += gs_dict[int((avg*69)/255)]
        f.write(line + '\n')
 
    # cleanup
    f.close()
    
    return output


In [173]:
def main(image, n_cols, scale):
    output_image = img_to_ascii(image, n_cols, scale)
    
    print('generating ASCII art...')
    
    # convert image to ascii txt
    outFile = 'out.txt'
 
    # open file
    f = open(outFile, 'w')
 
    # write to file
    for row in output_image:
        f.write(row + '\n')
 
    # cleanup
    f.close()
    print("ASCII art written to %s" % outFile)

In [174]:
def main2(image, n_cols, scale, output):
    img_to_ascii_to_txt(image, n_cols, scale, 'Output.txt')
    print('generating ASCII art...')
    print(f'ASCII art written to {output}')

In [175]:
main(images[1], n_cols=600, scale=0.43)

cols: 600, rows: 349
tile dims: 1.33 x 3.0930232558139537
generating ASCII art...
ASCII art written to out.txt


In [176]:
main2(images[1], n_cols=600, scale=0.43, output='output.txt')

generating ASCII art...
cols: 600, rows: 349
tile dims: 1.33 x 3.0930232558139537
ASCII art written to output.txt
